## Library Import

In [1]:
import pandas as pd
import random
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
# Seed 고정
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) 

In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [4]:
train

,ID,propertyType,bedrooms,latitude,longitude,suburbName,distanceMetro(km),distanceAirport(km),distanceHospital(km),distanceRailway(km),area(square_meters),monthlyRent(us_dollar)
0,Train_0000,Apartment,3,28.638710,77.295822,Delhi East,0.312579,22.646032,11.726966,7.352495,83.61,307
1,Train_0001,Independent Floor,1,28.498940,77.207191,Delhi South,2.486167,13.500583,7.527761,15.877066,83.61,110
2,Train_0002,Independent Floor,3,28.714123,77.154404,Delhi North,1.528794,18.918243,17.135939,10.315737,78.97,369
3,Train_0003,Independent Floor,3,28.704330,77.149956,Other,0.967121,17.749252,16.251937,9.797817,162.58,676
4,Train_0004,Apartment,4,28.577915,77.049446,Dwarka,0.834506,4.288189,15.541840,18.179806,218.32,418
...,...,...,...,...,...,...,...,...,...,...,...,...
8687,Train_8687,Apartment,1,28.602234,77.026001,Dwarka,0.005681,7.776390,18.212199,19.535831,46.45,159
8688,Train_8688,Apartment,1,28.644989,77.169296,Delhi Central,0.007987,12.969368,9.442664,5.039023,81.29,172
8689,Train_8689,Independent Floor,3,28.547377,77.259155,Delhi South,0.203502,17.094466,5.468956,11.109941,148.64,738
8690,Train_8690,Independent Floor,1,28.630501,77.277382,Delhi East,0.248603,20.628700,9.801128,5.679541,41.90,184


In [5]:
test

,ID,propertyType,bedrooms,latitude,longitude,suburbName,distanceMetro(km),distanceAirport(km),distanceHospital(km),distanceRailway(km),area(square_meters)
0,Test_0000,Independent House,1,28.644199,77.162407,Delhi Central,0.675741,12.480545,9.653289,5.706151,55.74
1,Test_0001,Independent Floor,3,28.724508,77.087051,West Delhi,1.992787,18.799390,21.086929,15.963967,88.26
2,Test_0002,Independent Floor,3,28.672693,77.103973,West Delhi,0.678442,13.190977,15.514042,11.916831,111.48
3,Test_0003,Independent Floor,1,28.645021,77.169235,Delhi Central,0.003422,12.968327,9.448341,5.045234,59.92
4,Test_0004,Independent Floor,1,28.607435,77.289627,Delhi East,0.299808,20.878677,9.180785,7.736478,27.87
...,...,...,...,...,...,...,...,...,...,...,...
8688,Test_8688,Apartment,3,28.586784,77.071671,Dwarka,1.060178,3.756045,13.500350,15.794588,148.64
8689,Test_8689,Independent Floor,3,28.646486,77.165459,Delhi Central,0.403543,12.863598,9.748500,5.426976,88.26
8690,Test_8690,Apartment,2,28.727852,77.086617,West Delhi,2.138730,19.169468,21.418370,16.214014,83.61
8691,Test_8691,Apartment,1,28.698778,77.145348,North Delhi,0.449243,17.014223,15.844727,9.709162,51.10


## Feature 추가
- 거리, 평균, 표준편차를 이용한 Feature

In [18]:
train["Distance mean"] = train.filter(regex="distance").mean(axis=1) 
train["Distance std"]  = train.filter(regex="distance").std(axis=1) 

test["Distance mean"] = test.filter(regex="distance").mean(axis=1)
test["Distance std"]  = test.filter(regex="distance").std(axis=1)

<hr>

## OneHotEncoding

In [21]:
from sklearn.preprocessing import OneHotEncoder

In [22]:
x_train = train.drop(columns=['ID', 'monthlyRent(us_dollar)'])
y_train = train['monthlyRent(us_dollar)']
x_test  = test.drop(columns=['ID'])

In [26]:
# qualitative column one-hot encoding
qual_col = ['propertyType','suburbName']
ohe = OneHotEncoder(sparse=False)

for i in qual_col:
    x_train = pd.concat([x_train, pd.DataFrame(ohe.fit_transform(x_train[[i]]), columns=ohe.categories_[0])], axis=1)
    
    for qual_value in np.unique(x_test[i]): 
        if qual_value not in np.unique(ohe.categories_): 
            ohe.categories_ = np.append(ohe.categories_, qual_value)
    # One Hot Encoder가 Test 데이터로부터 Fitting되는 것은 Data Leakage이므로, Test 데이터에는 Train 데이터로 Fitting된 One Hot Encoder로부터 transform만 수행되어야 합니다.
    x_test = pd.concat([x_test, pd.DataFrame(ohe.transform(x_test[[i]]), columns=ohe.categories_[0])], axis=1)
    
x_train = x_train.drop(qual_col, axis=1)
x_test = x_test.drop(qual_col, axis=1)
print('Done.')

Done.


<hr>

# Modeling

In [27]:
from supervised.automl import AutoML

In [28]:
automl = AutoML(mode="Compete", eval_metric='mae')

In [29]:
automl.fit(x_train, y_train)

AutoML directory: AutoML_8
The task is regression with evaluation metric mae
AutoML will use algorithms: ['Decision Tree', 'Linear', 'Random Forest', 'Extra Trees', 'LightGBM', 'Xgboost', 'CatBoost', 'Neural Network', 'Nearest Neighbors']
AutoML will stack models
AutoML will ensemble available models
AutoML steps: ['adjust_validation', 'simple_algorithms', 'default_algorithms', 'not_so_random', 'golden_features', 'kmeans_features', 'insert_random_feature', 'features_selection', 'hill_climbing_1', 'hill_climbing_2', 'boost_on_errors', 'ensemble', 'stack', 'ensemble_stacked']
* Step adjust_validation will try to check up to 1 model
1_DecisionTree mae 86.964401 trained in 0.3 seconds
Adjust validation. Remove: 1_DecisionTree
Validation strategy: 10-fold CV Shuffle
* Step simple_algorithms will try to check up to 4 models
1_DecisionTree mae 84.860879 trained in 2.47 seconds
2_DecisionTree mae 79.1883 trained in 2.14 seconds
3_DecisionTree mae 79.1883 trained in 2.17 seconds


In a future version, DataFrame.min(axis=None) will return a scalar min over the entire DataFrame. To retain the old behavior, use 'frame.min(axis=0)' or just 'frame.min()'
In a future version, DataFrame.min(axis=None) will return a scalar min over the entire DataFrame. To retain the old behavior, use 'frame.min(axis=0)' or just 'frame.min()'
In a future version, DataFrame.min(axis=None) will return a scalar min over the entire DataFrame. To retain the old behavior, use 'frame.min(axis=0)' or just 'frame.min()'
In a future version, DataFrame.min(axis=None) will return a scalar min over the entire DataFrame. To retain the old behavior, use 'frame.min(axis=0)' or just 'frame.min()'
In a future version, DataFrame.min(axis=None) will return a scalar min over the entire DataFrame. To retain the old behavior, use 'frame.min(axis=0)' or just 'frame.min()'
In a future version, DataFrame.min(axis=None) will return a scalar min over the entire DataFrame. To retain the old behavior, use 'frame.min

4_Linear mae 95.403088 trained in 3.98 seconds
* Step default_algorithms will try to check up to 7 models


In a future version, DataFrame.min(axis=None) will return a scalar min over the entire DataFrame. To retain the old behavior, use 'frame.min(axis=0)' or just 'frame.min()'
In a future version, DataFrame.min(axis=None) will return a scalar min over the entire DataFrame. To retain the old behavior, use 'frame.min(axis=0)' or just 'frame.min()'
In a future version, DataFrame.min(axis=None) will return a scalar min over the entire DataFrame. To retain the old behavior, use 'frame.min(axis=0)' or just 'frame.min()'
In a future version, DataFrame.min(axis=None) will return a scalar min over the entire DataFrame. To retain the old behavior, use 'frame.min(axis=0)' or just 'frame.min()'
In a future version, DataFrame.min(axis=None) will return a scalar min over the entire DataFrame. To retain the old behavior, use 'frame.min(axis=0)' or just 'frame.min()'
In a future version, DataFrame.min(axis=None) will return a scalar min over the entire DataFrame. To retain the old behavior, use 'frame.min

5_Default_LightGBM mae 56.28484 trained in 23.38 seconds


...

In a future version, DataFrame.min(axis=None) will return a scalar min over the entire DataFrame. To retain the old behavior, use 'frame.min(axis=0)' or just 'frame.min()'
In a future version, DataFrame.min(axis=None) will return a scalar min over the entire DataFrame. To retain the old behavior, use 'frame.min(axis=0)' or just 'frame.min()'
In a future version, DataFrame.min(axis=None) will return a scalar min over the entire DataFrame. To retain the old behavior, use 'frame.min(axis=0)' or just 'frame.min()'
In a future version, DataFrame.min(axis=None) will return a scalar min over the entire DataFrame. To retain the old behavior, use 'frame.min(axis=0)' or just 'frame.min()'
In a future version, DataFrame.min(axis=None) will return a scalar min over the entire DataFrame. To retain the old behavior, use 'frame.min(axis=0)' or just 'frame.min()'
In a future version, DataFrame.min(axis=None) will return a scalar min over the entire DataFrame. To retain the old behavior, use 'frame.min

12_Xgboost_Stacked mae 56.562011 trained in 76.42 seconds


In a future version, DataFrame.min(axis=None) will return a scalar min over the entire DataFrame. To retain the old behavior, use 'frame.min(axis=0)' or just 'frame.min()'


42_RandomForest_Stacked not trained. Stop training after the first fold. Time needed to train on the first fold 7.0 seconds. The time estimate for training on all folds is larger than total_time_limit.
* Step ensemble_stacked will try to check up to 1 model
Ensemble_Stacked mae 54.250615 trained in 14.34 seconds
AutoML fit time: 3634.15 seconds
AutoML best model: Ensemble_Stacked


AutoML(eval_metric='mae', mode='Compete')

In [30]:
idtest = test['ID']
idtest

0       Test_0000
1       Test_0001
2       Test_0002
3       Test_0003
4       Test_0004
          ...    
8688    Test_8688
8689    Test_8689
8690    Test_8690
8691    Test_8691
8692    Test_8692
Name: ID, Length: 8693, dtype: object

In [31]:
y_pred = automl.predict(x_test)

<hr>

# Submission

In [33]:
# Submission
submission = pd.read_csv('./sample_submission.csv')

In [34]:
submission['monthlyRent(us_dollar)'] = y_pred
submission

,ID,monthlyRent(us_dollar)
0,Test_0000,190.269038
1,Test_0001,247.620550
2,Test_0002,321.893779
3,Test_0003,178.595259
4,Test_0004,81.172495
...,...,...
8688,Test_8688,358.380489
8689,Test_8689,348.372913
8690,Test_8690,201.913400
8691,Test_8691,161.212373


In [35]:
submission.to_csv("AutoML_submission.csv", index = False)